# Threading for Reddit Scraping

In [2]:
from os.path import isfile
import praw
import pandas as pd
from time import sleep
import math

# Get credentials from DEFAULT instance in praw.ini
# Use your own tokens
reddit = praw.Reddit(
    client_id="tpkjzcHMAynTvw",
    client_secret="WdtCdrgnWMfZphmCgkl_fVFH-uA",
    user_agent="scraper",
    username="Whatchamacalit1",
    password="Tango9802!"
)


class SubredditScraper:

    def __init__(self, query, sort='new', lim=900, mode='w'):
        self.query = query
        self.sort = sort
        self.lim = lim
        self.mode = mode

        print(
            f'SubredditScraper instance created with values '
            f'query = {query}, sort = {sort}, lim = {lim}, mode = {mode}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit('all').search(self.query, sort='new', time_filter='month')
        elif self.sort == 'top':
            return self.sort, reddit.subreddit('all').search(self.query, sort='top', time_filter='month')
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit('all').search(self.query, sort='hot', time_filter='month')
        else:
            self.sort = 'relevance'
            print('Sort method was not recognized, defaulting to relevance.')
            return self.sort, reddit.subreddit('all').search(self.query, time_filter='month')

    def get_posts(self, printer=False):

        sub_dict = {
            'game_title': [], 'selftext': [], 'title': [], 'id': [], 'sorted_by': [],
            'num_comments': [], 'score': [], 'ups': [], 'downs': []}
        csv = f'game_titles.csv'

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        # Set csv_loaded to True if csv exists since you can't evaluate the
        # truth value of a DataFrame.
        df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

        print(f'csv = {csv}')
        print(f'After set_sort(), sort = {sort} and query = {self.query}')
        print(f'csv_loaded = {csv_loaded}')

        print(f'Collecting information from r/all for search term "{self.query}".')
        for post in subreddit:
            # Save any unique, non-stickied posts with descriptions to sub_dict.
            sub_dict['game_title'].append(self.query)
            sub_dict['selftext'].append(post.selftext)
            sub_dict['title'].append(post.title)
            sub_dict['id'].append(post.id)
            sub_dict['sorted_by'].append(sort)
            sub_dict['num_comments'].append(post.num_comments)
            sub_dict['score'].append(post.score)
            sub_dict['ups'].append(post.ups)
            sub_dict['downs'].append(post.downs)
            sleep(0.1)

        sub_dict_final = {
            'game_title': [self.query],  
            'score': [sum(sub_dict['score'])], 
            'ups': [sum(sub_dict['ups'])], 
            'downs': [sum(sub_dict['downs'])]}
        
        # print(sub_dict)
        new_df = pd.DataFrame(sub_dict_final)
        return new_df


#if __name__ == '__main__':
    #SubredditScraper('statistics', lim=997, mode='w', sort='new').get_posts()
    #SubredditScraper('Over Watch', lim=997, sort='new').get_posts()

In [7]:
def store_game_data(new_df):
    sleep(2)
    # Set csv_loaded to True if csv exists since you can't evaluate the
    # truth value of a DataFrame.
    csv = f'game_titles.csv'
    df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)
    
    # Store the new df
    if 'DataFrame' in str(type(df)):
        pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
        print(
            f'{len(new_df)} new posts collected and added to {csv}')
    else:
        new_df.to_csv(csv, index=False)
        print(f'{len(new_df)} posts collected and saved to {csv}')

In [4]:
import time
import threading

all_titles = pd.read_csv("All_Games.csv", error_bad_lines=False)
floor = math.floor

def split_by(k):
    size = all_titles.title.size
    split = math.floor(size/k)
    
    split_list = []
    
    for i in range(k-1):
        low = i*split
        high = (i+1)*split
        split_list.append([low,high])
    split_list.append([(k-1)*split,size])
    
    return split_list

lock = threading.Lock()
def scrape_game_range(index_low_high):
    low = index_low_high[0]
    high = index_low_high[1]
    index_range = range(low, high)
    
    for index in index_range:
        title = all_titles.title[index]
        
        lock.acquire()
        sleep(2)
        
        start = time.time()
        new_df = SubredditScraper('Over Watch', lim=997, sort='new').get_posts()
        end = time.time()
        print(end - start)
        
        lock.release()
        
        lock.acquire()
        
        start = time.time()
        store_game_data(new_df)
        end = time.time()
        print(end - start)
        
        lock.release()
        
split_by(5)

b'Skipping line 252: expected 8 fields, saw 9\nSkipping line 284: expected 8 fields, saw 10\nSkipping line 396: expected 8 fields, saw 9\nSkipping line 609: expected 8 fields, saw 9\nSkipping line 610: expected 8 fields, saw 9\nSkipping line 628: expected 8 fields, saw 9\nSkipping line 696: expected 8 fields, saw 9\nSkipping line 1561: expected 8 fields, saw 9\nSkipping line 2536: expected 8 fields, saw 9\nSkipping line 2650: expected 8 fields, saw 9\nSkipping line 2726: expected 8 fields, saw 9\nSkipping line 2773: expected 8 fields, saw 9\nSkipping line 2891: expected 8 fields, saw 11\nSkipping line 2892: expected 8 fields, saw 10\nSkipping line 2983: expected 8 fields, saw 9\nSkipping line 3005: expected 8 fields, saw 9\nSkipping line 3094: expected 8 fields, saw 10\nSkipping line 3119: expected 8 fields, saw 9\nSkipping line 3141: expected 8 fields, saw 10\nSkipping line 3151: expected 8 fields, saw 13\nSkipping line 3171: expected 8 fields, saw 17\nSkipping line 3289: expected 8 f

[[0, 1283], [1283, 2566], [2566, 3849], [3849, 5132], [5132, 6417]]

In [ ]:
import threading

for title in all_titles.title:
    print('Scraping: (' + str(count) + ')')
    count = count + 1
    SubredditScraper(str(title), lim=997, sort='new').get_posts()

Scraping: (1)
SubredditScraper instance created with values query = #killallzombies, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = #killallzombies
csv_loaded = 0
1 posts collected and saved to game_titles.csv
Scraping: (2)
SubredditScraper instance created with values query = 10 Second Ninja X, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = 10 Second Ninja X
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (3)
SubredditScraper instance created with values query = 100ft Robot Golf, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = 100ft Robot Golf
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (4)
SubredditScraper instance created with values query = 101 Ways to Die, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = 101 Ways to Die
csv_loaded = 1
1

1 new posts collected and added to game_titles.csv
Scraping: (26)
SubredditScraper instance created with values query = Absolver, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Absolver
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (27)
SubredditScraper instance created with values query = Abzû, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Abzû
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (28)
SubredditScraper instance created with values query = Ace Combat 7: Skies Unknown, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Ace Combat 7: Skies Unknown
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (29)
SubredditScraper instance created with values query = Achtung! Cthulhu Tactics, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort 

1 new posts collected and added to game_titles.csv
Scraping: (50)
SubredditScraper instance created with values query = Air Conflicts: Vietnam, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Air Conflicts: Vietnam
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (51)
SubredditScraper instance created with values query = Airship Q, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Airship Q
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (52)
SubredditScraper instance created with values query = AI: The Somnium Files, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = AI: The Somnium Files
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (53)
SubredditScraper instance created with values query = Akiba's Beat, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set

1 new posts collected and added to game_titles.csv
Scraping: (74)
SubredditScraper instance created with values query = Angry Birds Star Wars, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Angry Birds Star Wars
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (75)
SubredditScraper instance created with values query = Anodyne, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Anodyne
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (76)
SubredditScraper instance created with values query = Anoko wa Ore kara Hanarenai, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Anoko wa Ore kara Hanarenai
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (77)
SubredditScraper instance created with values query = Anomaly 2, sort = new, lim = 997, mode = w
csv = game_titles.csv
After 

1 new posts collected and added to game_titles.csv
Scraping: (99)
SubredditScraper instance created with values query = Assassin's Creed Chronicles: India, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Assassin's Creed Chronicles: India
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (100)
SubredditScraper instance created with values query = Assassin's Creed Chronicles: Russia, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Assassin's Creed Chronicles: Russia
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (101)
SubredditScraper instance created with values query = Assassin's Creed: The Ezio Collection, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Assassin's Creed: The Ezio Collection
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (102)
SubredditScraper ins

...

1 new posts collected and added to game_titles.csv
Scraping: (171)
SubredditScraper instance created with values query = Black Desert Online, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Black Desert Online
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (172)
SubredditScraper instance created with values query = Blackguards 2, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Blackguards 2
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (173)
SubredditScraper instance created with values query = Blackhole: Complete Edition, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Blackhole: Complete Edition
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (174)
SubredditScraper instance created with values query = Blackwood Crossing, sort = new, lim = 997, mode = w
csv = g

1 new posts collected and added to game_titles.csv
Scraping: (196)
SubredditScraper instance created with values query = Borderlands: The Handsome Collection, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Borderlands: The Handsome Collection
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (197)
SubredditScraper instance created with values query = Bound, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Bound
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (198)
SubredditScraper instance created with values query = Bound by Flame, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Bound by Flame
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (199)
SubredditScraper instance created with values query = Boundless, sort = new, lim = 997, mode = w
csv = game_titles.csv
Af

1 new posts collected and added to game_titles.csv
Scraping: (220)
SubredditScraper instance created with values query = Call of Cthulhu, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Call of Cthulhu
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (221)
SubredditScraper instance created with values query = Call of Duty: Advanced Warfare, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Call of Duty: Advanced Warfare
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (222)
SubredditScraper instance created with values query = Call of Duty: Black Ops III, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Call of Duty: Black Ops III
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (223)
SubredditScraper instance created with values query = Call of Duty: Black Ops IIII, sor

1 new posts collected and added to game_titles.csv
Scraping: (244)
SubredditScraper instance created with values query = Cel Damage HD, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Cel Damage HD
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (245)
SubredditScraper instance created with values query = Celeste, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Celeste
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (246)
SubredditScraper instance created with values query = Chaos;Child, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Chaos;Child
csv_loaded = 1
1 new posts collected and added to game_titles.csv
Scraping: (247)
SubredditScraper instance created with values query = Chaos Code: New Sign of Catastrophe, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort =

1 new posts collected and added to game_titles.csv
Scraping: (269)
SubredditScraper instance created with values query = Coffin Dodgers, sort = new, lim = 997, mode = w
csv = game_titles.csv
After set_sort(), sort = new and query = Coffin Dodgers
csv_loaded = 1
